In [1]:
import cv2
import numpy as np

def capture_image(filename="captured_face.jpg"):
    """ Capture an image from the webcam and save it """
    cap = cv2.VideoCapture(0)
    print("Press 's' to capture the image...")

    while True:
        ret, frame = cap.read()
        if not ret:
            continue

        cv2.imshow("Capture Image", frame)

        if cv2.waitKey(1) & 0xFF == ord('s'):  # Press 's' to save
            cv2.imwrite(filename, frame)
            print("Image saved successfully!")
            break

    cap.release()
    cv2.destroyAllWindows()
    return cv2.imread(filename, cv2.IMREAD_GRAYSCALE)

# Step 1: Capture the reference image
reference_image = capture_image()


Press 's' to capture the image...


error: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1301: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'


In [7]:
from sklearn.decomposition import PCA
import numpy as np
def extract_features(image, method="SIFT"):
    """ Extract features using SIFT, ORB, or BRIEF and return a flattened feature vector """
    if method == "SIFT":
        extractor = cv2.SIFT_create()
    elif method == "ORB":
        extractor = cv2.ORB_create()
    elif method == "BRIEF":
        fast = cv2.FastFeatureDetector_create()
        extractor = cv2.xfeatures2d.BriefDescriptorExtractor_create()
        keypoints = fast.detect(image, None)
        keypoints, descriptors = extractor.compute(image, keypoints)
        return keypoints, descriptors
    else:
        raise ValueError("Unsupported feature extraction method")

    keypoints, descriptors = extractor.detectAndCompute(image, None)

    if descriptors is not None:
        # Ensure PCA components are valid
        n_samples, n_features = descriptors.shape
        n_components = min(n_samples, n_features, 40)  # Dynamically choose max 20 features

        if n_components > 1:
            pca = PCA(n_components=n_components)
            descriptors = pca.fit_transform(descriptors)
        
        # Take mean of descriptors to form a single feature vector
        feature_vector = np.mean(descriptors, axis=0) if descriptors is not None else np.zeros(n_components)

        return keypoints, feature_vector
    else:
        return keypoints, None


In [8]:
# Step 2: Extract features from the captured image
method = "SIFT"  # Change to "ORB" or "BRIEF" to test
reference_keypoints, reference_features = extract_features(reference_image, method)


In [9]:
# Dictionary to store features
stored_features = {
    "method": method,
    "features": reference_features
}

print(f"Stored feature vector using {method}: {reference_features.shape} dimensions")


Stored feature vector using SIFT: (40,) dimensions


In [6]:
from scipy.spatial.distance import euclidean
from sklearn.metrics.pairwise import cosine_similarity

def match_features(new_features, stored_features, method="euclidean"):
    """ Compare features using Euclidean Distance or Cosine Similarity """
    if method == "euclidean":
        distance = euclidean(new_features, stored_features)
        return distance  # Lower value means closer match
    elif method == "cosine":
        similarity = cosine_similarity([new_features], [stored_features])[0][0]
        return similarity  # Higher value means closer match
    else:
        raise ValueError("Unsupported matching method")

# Capture a new image for comparison
comparison_image = capture_image(filename="comparison_face.jpg")

# Extract features from the new image
comparison_keypoints, comparison_features = extract_features(comparison_image, method)

# Compare using Euclidean Distance
euclidean_distance = match_features(comparison_features, stored_features["features"], method="euclidean")
print(f"Euclidean Distance: {euclidean_distance}")

# Compare using Cosine Similarity
cosine_score = match_features(comparison_features, stored_features["features"], method="cosine")
print(f"Cosine Similarity Score: {cosine_score}")


Press 's' to capture the image...
Image saved successfully!
Euclidean Distance: 1.249654724233551e-05
Cosine Similarity Score: -0.2074752300977707
